# 단일 PDF 파일 챗봇 샘플 코드

#### Amazon Bedrock의 Knowledge Base 기능 (Chat with your document) 활용
- 이 기능은 현재 Claude 3 Sonnet 과 Claude 3 Haiku 만 지원하고 있습니다.
- 업로드 되는 문서는 10MB 이내여야 합니다.

In [1]:
!pip install boto3 -U

In [2]:
import boto3
import json
import time
import base64

# Bedrock 클라이언트 생성
bedrock_client = boto3.client('bedrock-agent-runtime')


In [3]:
# 문서 파일 경로 

file_name = "Steel Business Briefing_04 Jul 2024.pdf"

document_uri = f"s3://240605-jesamkim-test-bucket/rag-doc-1/{file_name}"  # S3에 PDF를 업로드 한 경우


In [4]:
document_uri

's3://240605-jesamkim-test-bucket/rag-doc-1/Steel Business Briefing_04 Jul 2024.pdf'

In [5]:
# 사용자 쿼리
user_query = """
철강 산업 동향에 대해 자세히 알려주세요.
"""

prompt_template = f"""
You are a question answering agent. 
I will provide you with a set of search results. 
The user will provide you with a question. 
Your job is to answer the user's question using only information from the search results. 
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. 
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$

$output_format_instructions$

User Question : {user_query}
"""


In [6]:
%%time

def retrieveAndGenerate(input_text, sourceType, document_s3_uri=None, local_sources=None):
    model_arn = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0" # 현재 KB에서는 Claude 3 Sonnet 까지만 지원됩니다.

    if sourceType == "S3":
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_s3_uri  
                            }
                        }
                    ]
                }
            }
        )
        
    else:
        return 1

response = retrieveAndGenerate(
    input_text=prompt_template,
    sourceType="S3",              
    document_s3_uri=document_uri, 
    )


# 응답 처리
output = response.get('output', {})
generated_text = output.get('text', '')
print("Generated response: \n", generated_text)

Generated response: 
 철강 산업의 최근 동향은 다음과 같습니다:

- 아시아 지역에서 열연코일 시장은 정체 상태이며, 구매 관심은 여전히 부족한 편입니다. 중국산 열연코일은 일본과 한국산에 비해 가격 경쟁력이 낮아지고 있습니다. (출처: 11, 12)

- 유럽 열연코일 가격은 여름 시즌을 앞두고 평평한 수준을 유지하고 있습니다. 수요 부족으로 인해 활동 수준이 낮은 상황입니다. (출처: 19, 20)

- 터키 철근 수출 가격은 수출 시장의 약세로 인해 좁은 범위에서 거래되고 있습니다. (출처: 22, 23)

- 중국 철강 시장 전망은 부동산 부문의 약세로 인해 단기적으로 부진할 것으로 예상됩니다. 정부의 경기 부양책이 있겠지만 효과는 제한적일 것으로 보입니다. (출처: 29, 30, 31, 32, 33, 34)
CPU times: user 13 ms, sys: 3.5 ms, total: 16.5 ms
Wall time: 24.2 s


In [7]:
# 인용 정보 출력 (있는 경우)
citations = response.get('citations', [])
if citations:
    print("\nCitations:\n")
    for citation in citations:
        generated_response_part = citation.get('generatedResponsePart', {})
        text_response_part = generated_response_part.get('textResponsePart', {})
        span = text_response_part.get('span', {})
        start = span.get('start', 'N/A')
        end = span.get('end', 'N/A')
        text = text_response_part.get('text', 'N/A')

        #print(f"- Span: {start}-{end}")
        #print(f"  Text: {text}")

        retrieved_references = citation.get('retrievedReferences', [])
        for ref in retrieved_references:
            content = ref.get('content', {})
            text = content.get('text', 'N/A')
            location = ref.get('location', {})
            s3_location = location.get('s3Location', {})
            uri = s3_location.get('uri', 'N/A')

            print(f"  Reference:")
            print(f"    URI: {uri}")
            print(f"    Content: {text[:200]}...")  # 처음 200자만 출력
        print()  # 각 citation 사이에 빈 줄 추가


Citations:

  Reference:
    URI: s3://240605-jesamkim-test-bucket/rag-doc-1/Steel Business Briefing_04 Jul 2024.pdf
    Content: All rights reserved. 2   SBB Daily Briefing July 4, 2024   Asia   Flat Products   Asian HRC at standstill; buying indications   lagging behind  „ Bid-ask spread still about $10/mt    „ Chinese HRC los...
  Reference:
    URI: s3://240605-jesamkim-test-bucket/rag-doc-1/Steel Business Briefing_04 Jul 2024.pdf
    Content: Chinese export offers have gradually lost price  competitiveness over Japanese and South Korean materials, with  indicative offers for the latter two heard at $530/mt FOB Japan  and $520/mt FOB South ...
  Reference:
    URI: s3://240605-jesamkim-test-bucket/rag-doc-1/Steel Business Briefing_04 Jul 2024.pdf
    Content: Platts is part of S&P Global Commodity Insights. — Jia hui Tan, Shuocheng Ni   EMEA   Flat Products   European HRC prices remains flat   ahead of a quiet expected summer  „ Demand to stay weak until S...
  Reference:
    URI: